In [3]:
print(os.getpid())

472138


In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "7"
os.environ["OMP_NUM_THREADS"] = "7"
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  StandardScaler
from sklearn.decomposition import PCA
import gc
import numpy as np
base_dir = "/home/skovtun/Python_projects/Kaggle/Single cell"
data_dir = os.path.join(base_dir, "Single_cell_data")
random_state=77
os.chdir("/home/skovtun/Python_projects/Kaggle/Single cell")
metadata_old = pd.read_csv('Single_cell_data/metadata.csv', index_col = 'cell_id')
fix = pd.read_csv('Single_cell_data/metadata_cite_day_2_donor_27678.csv', index_col = 'cell_id')
metadata = pd.concat([metadata_old, fix], axis = 0)
del fix, metadata_old
#cite = pd.read_hdf('Single_cell_data/train_cite_inputs.h5')

In [ ]:
# splitting site to 7 groups based on cell type.
# cite_metadata = metadata[metadata['technology'] == 'citeseq']
# cell_types = cite_metadata['cell_type'].unique()
# base_dir = "/home/skovtun/Python_projects/Kaggle/Single cell"
# data_dir = os.path.join(base_dir, "Single_cell_data")

# os.chdir(data_dir)

# for cell_type in cell_types:
#     print(f"Processing {cell_type}...")
#     idx = cite_metadata[cite_metadata['cell_type'] == cell_type].index
#     valid_ids = idx.intersection(cite.index)
#     subset = cite.loc[valid_ids]

#     # write directly to file under its own key
#     subset.to_parquet(
#     f"X_{cell_type}.parquet",
#     engine="pyarrow",
#     index=True,
#     coerce_timestamps="ms",
#     allow_truncated_timestamps=True,
#     version="2.6")
#     # free memory immediately
#     del subset
# os.chdir(base_dir)
# import gc; gc.collect()

In [10]:
#splitting cite test to 7 groups based on cell type.
# fix has more columns than initial test file, so I am cutting the columns so the datasets 
#whoud be aligned
# import time
# evaluation_old = pd.read_hdf('Single_cell_data/test_cite_inputs.h5')
# fix = pd.read_hdf('Single_cell_data/test_cite_inputs_day_2_donor_27678.h5')
# common_cols = evaluation_old.columns
# fix_aligned = fix[common_cols]
# cite_evaluation = pd.concat([evaluation_old,fix_aligned], axis = 0)
# del evaluation_old, fix
# cite_metadata = metadata[metadata['technology'] == 'citeseq']
# cell_types = cite_metadata['cell_type'].unique()
# base_dir = "/home/skovtun/Python_projects/Kaggle/Single cell"
# data_dir = os.path.join(base_dir, "Single_cell_data")

# os.chdir(data_dir)
# for cell_type in cell_types:
#     print(f"Processing {cell_type}...")
#     start_time = time.time()
#     idx = cite_metadata[cite_metadata['cell_type'] == cell_type].index
#     valid_ids = idx.intersection(cite_evaluation.index)
#     subset = cite_evaluation.loc[valid_ids]

#     # write directly to file under its own key
#     subset.to_parquet(
#     f"X_eval{cell_type}.parquet",
#     engine="pyarrow",
#     index=True,
#     coerce_timestamps="ms",
#     allow_truncated_timestamps=True,
#     version="2.6")
#     print((time.time() - start_time)/60)
#     # free memory immediately
#     del subset

# os.chdir(base_dir)
# import gc; gc.collect()

Processing HSC...
0.24375638564427693
Processing EryP...
0.18212833007176718
Processing NeuP...
0.15617162386576336
Processing MasP...
0.15948372681935627
Processing MkP...
0.1079578955968221
Processing BP...
0.035975893338521324
Processing MoP...
0.048778629302978514


0

BP has only ~150 cells in both train and evaluation sets, so I just add BP cells I need to evaluate to my HSC model. 

In [8]:
os.chdir(data_dir)
targets = pd.read_hdf('train_cite_targets.h5') #all type of cells
inputs1 = pd.read_parquet('X_HSC.parquet')
inputs2 = pd.read_parquet('X_MasP.parquet')
inputs3 = pd.read_parquet('X_MkP.parquet')
inputs4 = pd.read_parquet('X_MoP.parquet')
inputs5 = pd.read_parquet('X_NeuP.parquet')
inputs6 = pd.read_parquet('X_BP.parquet')
inputs7 = pd.read_parquet('X_EryP.parquet')

inputs = pd.concat([inputs1,inputs2,inputs3,inputs4,inputs5,inputs6, inputs7], axis = 0)
del inputs1,inputs2,inputs3,inputs4,inputs5,inputs6,inputs7

evaluation1 = pd.read_parquet('X_evalHSC.parquet')
evaluation2 = pd.read_parquet('X_evalMasP.parquet')
evaluation3 = pd.read_parquet('X_evalMkP.parquet')
evaluation4 = pd.read_parquet('X_evalMoP.parquet')
evaluation5 = pd.read_parquet('X_evalNeuP.parquet')
evaluation6 = pd.read_parquet('X_evalBP.parquet')
evaluation7 = pd.read_parquet('X_evalEryP.parquet')
evaluation = pd.concat([evaluation1,evaluation2,evaluation3,evaluation4,evaluation5,evaluation6,evaluation7], axis=0)

del evaluation1,evaluation2,evaluation3,evaluation4,evaluation5,evaluation6,evaluation7

Get relevant columns for all targets using external file from https://www.biolegend.com/en-us/totalseq/barcode-lookup. The rest will be compressed using PCA.

In [3]:
os.chdir(data_dir)
#Getting info about gene barcodes relevant to proteins from an external file and choosing these colums
# to get corresponding columns from input data to use as is in modeling. The rest will be compressed 
# using pca
targets = pd.read_hdf('train_cite_targets.h5') #all type of cells
barcode = pd.read_csv('Totalseq_a.csv')
proteins = targets.columns
all_protein_barcodes = barcode[barcode['Description'].isin(proteins)].sort_values('Description')
h_protein_barcodes = all_protein_barcodes[all_protein_barcodes['Reactivity'].str.contains('Human')]
ens_gene_ids = list(h_protein_barcodes['Ensembl Gene Id'].unique())
del barcode
clean_input_cols = [c.split('_')[0] for c in inputs.columns]
eval_input_cols = [c.split('_')[0] for c in evaluation.columns]
inputs.columns = clean_input_cols
evaluation.columns = eval_input_cols
common_cols = list(set(ens_gene_ids)&set(inputs.columns))
pca_cols = list(set(inputs.columns) - set(common_cols))
assert(clean_input_cols == eval_input_cols)

NameError: name 'inputs' is not defined

In [4]:
len(ens_gene_ids)

128

In [10]:
#indexes of the cell type we are working
eval_idx = evaluation.index
idx = inputs.index
# choosing the same rows form targets
valid_ids = idx.intersection(targets.index)
y = targets.loc[valid_ids]
# random split to train_test
rng = np.random.default_rng(77)
shuffled_idx = rng.permutation(valid_ids)
split_ratio = 0.8
split_point = int(len(shuffled_idx) * split_ratio)
train_idx = shuffled_idx[:split_point]
test_idx = shuffled_idx[split_point:]
inputs_train, inputs_test = inputs.loc[train_idx], inputs.loc[test_idx]
inputs_train_to_compress = inputs_train[pca_cols]
inputs_train_orig = inputs_train[common_cols]
inputs_test_to_compress = inputs_test[pca_cols]
inputs_test_orig = inputs_test[common_cols]
y_train, y_test = y.loc[train_idx], y.loc[test_idx]

eval_orig = evaluation[common_cols]
eval_to_compress = evaluation[pca_cols]

In [11]:
# to save memory, deleting targets, inputs, evaluation
del targets, inputs, evaluation

In [12]:
scaler = StandardScaler()
inputs_train_scaled = scaler.fit_transform(inputs_train_to_compress)
del inputs_train_to_compress
print('inputs_train scaled')
inputs_test_scaled = scaler.transform(inputs_test_to_compress)
del inputs_test_to_compress
print('inputs_test scaled')
evaluation_scaled = scaler.transform(eval_to_compress)
del eval_to_compress
print('inputs_val scaled')




inputs_train scaled
inputs_test scaled
inputs_val scaled


In [13]:
import time
start_time = time.time()
pca = PCA(n_components=200)
inputs_train_pca = pca.fit_transform(inputs_train_scaled)
print('inputs_train_pca completed')
print((time.time() - start_time)/60)
start_time = time.time()
inputs_test_pca = pca.transform(inputs_test_scaled)
print('inputs_test_pca completed')
print((time.time() - start_time)/60)
start_time = time.time()
evaluation_pca = pca.transform(evaluation_scaled)
print('evaluation_pca completed')
print((time.time() - start_time)/60)

inputs_train_pca completed
0.5527858098347982
inputs_test_pca completed
0.008547735214233399
evaluation_pca completed
0.061624594529469806


In [14]:
inputs_train_pca.shape

(56790, 200)

In [15]:
inputs_train_all = np.hstack([inputs_train_orig.values, inputs_train_pca])
inputs_test_all = np.hstack([inputs_test_orig.values, inputs_test_pca])
evaluation_all = np.hstack([eval_orig.values, evaluation_pca])

Dealing with mixture of original and PCA transformed columns, it's impossible to remove donor effect. I tried linear and non-linear method, but the signal is very strong.

In [16]:
pca_cols = [f'PC{i+1}' for i in range(inputs_train_pca.shape[1])]
all_cols = list(inputs_train_orig.columns) + pca_cols
print(len(all_cols))   
X_train_all = pd.DataFrame(inputs_train_all, index = train_idx, columns = all_cols)
X_test_all = pd.DataFrame(inputs_test_all, index = test_idx, columns = all_cols)
X_eval_all = pd.DataFrame(evaluation_all, index = eval_idx, columns = all_cols)

303


In [17]:
metadata.columns

Index(['day', 'donor', 'cell_type', 'technology'], dtype='object')

In [18]:
X_train = pd.merge(X_train_all, metadata[['day','donor','cell_type']], how = 'left', left_index = True, right_index = True)
X_test = pd.merge (X_test_all, metadata[['day','donor','cell_type']], how = 'left', left_index = True, right_index = True)
X_eval = pd.merge (X_eval_all, metadata[['day','donor','cell_type']], how = 'left', left_index = True, right_index = True)

In [25]:
X_train['cell_type'] = X_train['cell_type'].astype('category')
X_test['cell_type'] = X_test['cell_type'].astype('category') 
X_eval['cell_type'] = X_eval['cell_type'].astype('category') 

X_train_one=pd.get_dummies(X_train, columns=['cell_type'], prefix = 'ct', dummy_na=False)
X_test_one=pd.get_dummies(X_test, columns=['cell_type'], prefix = 'ct', dummy_na=False)
X_eval_one=pd.get_dummies(X_eval, columns=['cell_type'], prefix = 'ct', dummy_na=False)



In [34]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# X_train: (n_cells, 301) PCA or features
# Y_protein: (n_cells, 140) protein targets as a DataFrame or ndarray
# protein_names: length-140 list/Index of protein IDs

protein_names = y_train.columns  # if DataFrame

stats = []

ridge = Ridge(alpha=1.0)

for j, protein in enumerate(protein_names):
    y = y_train[protein].values  # shape (n_cells,)

    var = np.var(y)
    mean = np.mean(y)

    # quick baseline difficulty measure
    r2 = cross_val_score(
        ridge, X_train_one,y_train[protein],
        cv=3,
        scoring="r2",
        n_jobs=-1
    ).mean()

    stats.append({"protein": protein, "var": var, "mean": mean, "r2": r2})

stats_df = pd.DataFrame(stats)
stats_df["var_bin"] = pd.qcut(stats_df["var"], 3, labels=["low_var", "mid_var", "high_var"])
stats_df["r2_bin"]  = pd.qcut(stats_df["r2"],  3, labels=["hard", "medium", "easy"])

rep_proteins = []

for v in ["low_var", "mid_var", "high_var"]:
    for r in ["hard", "medium", "easy"]:
        block = stats_df[(stats_df["var_bin"] == v) & (stats_df["r2_bin"] == r)]
        if len(block) == 0:
            continue
        # pick one protein in this cell – e.g. closest to median r2
        chosen1 = block.iloc[(block["r2"] - block["r2"].median()).abs().argmin()]
        rep_proteins.append(chosen1["protein"])
        chosen2 = block.iloc[(block["r2"] - block["r2"].median()).abs().argmax()]
        rep_proteins.append(chosen2["protein"])
rep_proteins = list(dict.fromkeys(rep_proteins))  # unique, preserve order

In [35]:
rep_proteins

['Mouse-IgG2a',
 'CD16',
 'CD270',
 'CD141',
 'CD21',
 'CD93',
 'CD86',
 'CD272',
 'CD304',
 'CD142',
 'CD7',
 'CD11c',
 'CD22',
 'CD44',
 'CD41']

In [36]:
# XGB global model (all cells) + cell_type dummies
# Use with xgb.train + early stopping (as in your loop)

candidate_params = [
    # A) strong default (robust under donor shift)
    dict(max_depth=5, min_child_weight=40, subsample=0.80, colsample_bytree=0.40,
         learning_rate=0.05, reg_alpha=0.0, reg_lambda=4.0, gamma=0.0),

    # B) slightly shallower, more conservative
    dict(max_depth=4, min_child_weight=60, subsample=0.75, colsample_bytree=0.45,
         learning_rate=0.05, reg_alpha=0.0, reg_lambda=5.0, gamma=0.0),

    # C) deeper but controlled by min_child_weight + sampling
    dict(max_depth=6, min_child_weight=50, subsample=0.75, colsample_bytree=0.35,
         learning_rate=0.04, reg_alpha=0.0, reg_lambda=5.0, gamma=0.0),

    # D) strong column subsampling (often helps with correlated PCA/LSI + OHE)
    dict(max_depth=5, min_child_weight=50, subsample=0.70, colsample_bytree=0.30,
         learning_rate=0.05, reg_alpha=0.0, reg_lambda=5.0, gamma=0.0),

    # E) add some L1 sparsity (useful when many weak features)
    dict(max_depth=5, min_child_weight=40, subsample=0.80, colsample_bytree=0.40,
         learning_rate=0.05, reg_alpha=0.3, reg_lambda=4.0, gamma=0.0),

    # F) extra conservative (good baseline when overfitting is suspected)
    dict(max_depth=4, min_child_weight=80, subsample=0.70, colsample_bytree=0.35,
         learning_rate=0.06, reg_alpha=0.1, reg_lambda=6.0, gamma=0.0),

    # G) slightly higher capacity but regularized
    dict(max_depth=6, min_child_weight=60, subsample=0.80, colsample_bytree=0.40,
         learning_rate=0.03, reg_alpha=0.0, reg_lambda=6.0, gamma=0.0),

    # H) DART option (only if runtime allows; can improve generalization)
    dict(booster="dart", max_depth=5, min_child_weight=50, subsample=0.80, colsample_bytree=0.35,
         learning_rate=0.05, reg_alpha=0.0, reg_lambda=5.0, gamma=0.0,
         rate_drop=0.10, skip_drop=0.50),
]


In [37]:




def kaggle_mean_cellwise_pearson(y_true, y_pred):
    """
    Mean per-cell Pearson correlation with the rule that constant prediction 
    should give -1 in correlation
    """
    yt = np.asarray(y_true)
    yp = np.asarray(y_pred)
    # constant prediction per row -> -1
    const_pred = np.all(yp == yp[:, [0]], axis=1)
    y_pred_centered = yp - yp.mean(axis=1, keepdims=True)
    y_true_centered = yt - yt.mean(axis=1, keepdims=True)
    num = np.sum(y_true_centered * y_pred_centered, axis=1)
    den = np.sqrt(np.sum(y_true_centered **2, axis=1) * np.sum(y_pred_centered**2, axis=1))
    corrs = np.empty(y_true.shape[0], dtype=float)
    corrs[const_pred] = -1.0
    # Pearson can be calculated
    valid = (~const_pred) & (den > 0)
    corrs[valid] = num[valid] / den[valid]
    # non-constant with zero denominator
    corrs[~const_pred & (den == 0)] = np.nan
    return np.nanmean(corrs)  
    
def split_indices_donor(X_df, donor_col="donor"):
    """
    Randomly choose one donor as validation donor.
    Split all rows belonging to that donor into validation set;
    all remaining rows into training.
    """
    donor = X_df[donor_col].to_numpy()
    uniq = np.unique(donor)

    # choose a random donor reproducibly
    rng = np.random.RandomState(77)
    val_donor = rng.choice(uniq)

    # select indices
    train_idx = np.where(donor != val_donor)[0]
    val_idx   = np.where(donor == val_donor)[0]

    return train_idx, val_idx, val_donor

def to_float32_numpy(df):
    """
    Convert dataframe to float32 numpy (xgb likes float32)
    """
    arr = df.to_numpy()
    if arr.dtype != np.float32:
        arrn = arr.astype(np.float32, copy=False)
    return arrn

def predict_best(booster, dmat):
    """
    Predict with a Booster using the best number of trees based on early stopping.
    """
    attrs = booster.attributes() 
    ntree = int(attrs["best_iteration"]) + 1
    if ntree is not None:
        return booster.predict(dmat, iteration_range=(0, ntree))
    return booster.predict(dmat)


In [38]:
# ============================================================
# XGBoost global-parameter tuning for ONE cell type
# - Donor used ONLY for splitting 
# - Train one model per protein in rep_proteins
# - Score each candidate param set by Kaggle metric proxy:
#       mean over cells of Pearson corr across proteins
# - Uses xgboost.train + DMatrix
#
# ============================================================

import numpy as np
import xgboost as xgb

base_params = dict(
    objective="reg:squarederror",
    eval_metric="rmse",
    tree_method="hist",
    seed=77,
    verbosity=0,
)


tr_idx, va_idx, used_val_donor = split_indices_donor(X_train_one, donor_col="donor")

X_feat = X_train_one.drop(columns=['donor']) 
X_tr = to_float32_numpy(X_feat.iloc[tr_idx])
X_va = to_float32_numpy(X_feat.iloc[va_idx])


Y_tr = y_train.loc[X_feat.index[tr_idx], rep_proteins].to_numpy(dtype=np.float32, copy=False)
Y_va = y_train.loc[X_feat.index[va_idx], rep_proteins].to_numpy(dtype=np.float32, copy=False)

dtrain = xgb.DMatrix(X_tr)
dvalid = xgb.DMatrix(X_va)

scores = []

print(f"Donor used for split: {used_val_donor}")
print(f"Train cells: {len(tr_idx)}  Val cells: {len(va_idx)}  #rep_proteins: {len(rep_proteins)}")
print(f"#features used (donor excluded): {X_tr.shape[1]}")

for k, cand in enumerate(candidate_params, 1):
    params = {**base_params, **cand}
    preds = np.zeros_like(Y_va, dtype=np.float32)

    # train one booster per protein (same params)
    for j in range(len(rep_proteins)):
        dtrain.set_label(Y_tr[:, j])
        dvalid.set_label(Y_va[:, j])

        booster = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=20000,
            evals=[(dvalid, "valid")],
            early_stopping_rounds= 200,
            verbose_eval=False,
        )

        preds[:, j] = predict_best(booster, dvalid)

    score = kaggle_mean_cellwise_pearson(Y_va, preds)
    scores.append({"cand": cand, "score": score})

    print(f"{k:02d}/{len(candidate_params)}  mean_cellwise_corr={score:.5f}  cand={cand}")

best = max(scores, key=lambda d: d["score"])

print("\nBEST XGB CANDIDATE (global params for this cell type):")
print(f"mean_cellwise_corr={best['score']:.5f}")
print(best["cand"])



Donor used for split: 13176
Train cells: 39034  Val cells: 17756  #rep_proteins: 15
#features used (donor excluded): 311
01/8  mean_cellwise_corr=0.89871  cand={'max_depth': 5, 'min_child_weight': 40, 'subsample': 0.8, 'colsample_bytree': 0.4, 'learning_rate': 0.05, 'reg_alpha': 0.0, 'reg_lambda': 4.0, 'gamma': 0.0}
02/8  mean_cellwise_corr=0.89853  cand={'max_depth': 4, 'min_child_weight': 60, 'subsample': 0.75, 'colsample_bytree': 0.45, 'learning_rate': 0.05, 'reg_alpha': 0.0, 'reg_lambda': 5.0, 'gamma': 0.0}
03/8  mean_cellwise_corr=0.89831  cand={'max_depth': 6, 'min_child_weight': 50, 'subsample': 0.75, 'colsample_bytree': 0.35, 'learning_rate': 0.04, 'reg_alpha': 0.0, 'reg_lambda': 5.0, 'gamma': 0.0}
04/8  mean_cellwise_corr=0.89849  cand={'max_depth': 5, 'min_child_weight': 50, 'subsample': 0.7, 'colsample_bytree': 0.3, 'learning_rate': 0.05, 'reg_alpha': 0.0, 'reg_lambda': 5.0, 'gamma': 0.0}
05/8  mean_cellwise_corr=0.89872  cand={'max_depth': 5, 'min_child_weight': 40, 'subsam

KeyboardInterrupt: 

In [39]:
X_train = X_train_one.drop('donor', axis = 1)
X_test =  X_test_one.drop('donor', axis =1)
X_eval = X_eval_one.drop('donor', axis = 1)

The leaderboard metric checks:

*For each cell, the pattern across all proteins must be correct.*

That means each single-target model should produce predictions that are well correlated with the true values across cells — i.e., the variation between cells should match reality

In [40]:
import xgboost as xgb
import pandas as pd
import numpy as np

y_pred = pd.DataFrame(index=y_test.index, columns=y_test.columns, dtype=np.float32)
y_pred_train = pd.DataFrame(index=y_train.index, columns=y_train.columns, dtype=np.float32)
y_pred_eval = pd.DataFrame(index=X_eval.index, columns=y_train.columns, dtype=np.float32)

Xtr = to_float32_numpy(X_train)
Xte = to_float32_numpy(X_test)
Xev = to_float32_numpy(X_eval)

dtrain = xgb.DMatrix(Xtr)
dtest  = xgb.DMatrix(Xte)
deval  = xgb.DMatrix(Xev)

xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",   
    "eta": 0.05,
    "max_depth": 5,
    "min_child_weight": 40,
    "subsample": 0.88,
    "colsample_bytree": 0.4,
    "gamma": 0.0,
    "reg_lambda": 4.0,
    "reg_alpha": 0.3,
    "seed": 77,
    "verbosity": 0,
}
i = 1
for protein in y_train.columns:
    print(i, protein)

    ytr = y_train[protein].to_numpy(dtype=np.float32)
    dtrain.set_label(ytr)

    booster = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=600,
        verbose_eval=False,
    )

    y_pred_train[protein] = booster.predict(dtrain)
    y_pred[protein]       = booster.predict(dtest)
    y_pred_eval[protein]  = booster.predict(deval)

    i += 1

print("Done.")
score = kaggle_mean_cellwise_pearson(y_train, y_pred_train)
print("Kaggle Pierson correlation train:", score)
score = kaggle_mean_cellwise_pearson(y_test, y_pred)
print("Kaggle Pierson correlation train:", score)


1 CD86
2 CD274
3 CD270
4 CD155
5 CD112
6 CD47
7 CD48
8 CD40
9 CD154
10 CD52
11 CD3
12 CD8
13 CD56
14 CD19
15 CD33
16 CD11c
17 HLA-A-B-C
18 CD45RA
19 CD123
20 CD7
21 CD105
22 CD49f
23 CD194
24 CD4
25 CD44
26 CD14
27 CD16
28 CD25
29 CD45RO
30 CD279
31 TIGIT
32 Mouse-IgG1
33 Mouse-IgG2a
34 Mouse-IgG2b
35 Rat-IgG2b
36 CD20
37 CD335
38 CD31
39 Podoplanin
40 CD146
41 IgM
42 CD5
43 CD195
44 CD32
45 CD196
46 CD185
47 CD103
48 CD69
49 CD62L
50 CD161
51 CD152
52 CD223
53 KLRG1
54 CD27
55 CD107a
56 CD95
57 CD134
58 HLA-DR
59 CD1c
60 CD11b
61 CD64
62 CD141
63 CD1d
64 CD314
65 CD35
66 CD57
67 CD272
68 CD278
69 CD58
70 CD39
71 CX3CR1
72 CD24
73 CD21
74 CD11a
75 CD79b
76 CD244
77 CD169
78 integrinB7
79 CD268
80 CD42b
81 CD54
82 CD62P
83 CD119
84 TCR
85 Rat-IgG1
86 Rat-IgG2a
87 CD192
88 CD122
89 FceRIa
90 CD41
91 CD137
92 CD163
93 CD83
94 CD124
95 CD13
96 CD2
97 CD226
98 CD29
99 CD303
100 CD49b
101 CD81
102 IgD
103 CD18
104 CD28
105 CD38
106 CD127
107 CD45
108 CD22
109 CD71
110 CD26
111 CD115
112 CD63

In [41]:
#Final run
X = pd.concat([X_train,X_test], axis = 0)
y = pd.concat([y_train,y_test], axis = 0)
# Convert once (XGBoost prefers float32)
Xtr = to_float32_numpy(X)
Xev = to_float32_numpy(X_eval)

dtrain = xgb.DMatrix(Xtr)
deval  = xgb.DMatrix(Xev)

y_pred = pd.DataFrame(index=y.index, columns=y.columns, dtype=np.float32)
y_pred_eval = pd.DataFrame(index=X_eval.index, columns=y_train.columns, dtype=np.float32)

NUM_BOOST_ROUND = 600   

i = 1
for protein in y_train.columns:
    print(i, protein)
    start_time = time.time()
    ytr = y[protein].to_numpy(dtype=np.float32)
    dtrain.set_label(ytr)

    booster = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=NUM_BOOST_ROUND,
        verbose_eval=False,
    )

    y_pred_eval[protein]  = booster.predict(deval)
    print((time.time() - start_time)/60)
    i += 1

print("Done.")


1 CD86
0.28791080713272094
2 CD274
0.3140679637591044
3 CD270
0.32285065253575645
4 CD155
0.32880709568659466
5 CD112
0.343480118115743
6 CD47
0.33845674991607666
7 CD48
0.33193631172180177
8 CD40
0.2993111173311869
9 CD154
0.3256821910540263
10 CD52
0.3157497485478719
11 CD3
0.317780876159668
12 CD8
0.31485631863276164
13 CD56
0.31736618677775064
14 CD19
0.28911630709966024
15 CD33
0.34928067922592165
16 CD11c
0.24606310923894245
17 HLA-A-B-C
0.34258826573689777
18 CD45RA
0.3456883509953817
19 CD123
0.338648521900177
20 CD7
0.29847044150034585
21 CD105
0.3270050843556722
22 CD49f
0.3313333710034688
23 CD194
0.31337769826253253
24 CD4
0.3154712359110514
25 CD44
0.33819146553675333
26 CD14
0.28433096011479697
27 CD16
0.313196341196696
28 CD25
0.2972115238507589
29 CD45RO
0.323922860622406
30 CD279
0.3123837033907572
31 TIGIT
0.32310200134913125
32 Mouse-IgG1
0.3210986256599426
33 Mouse-IgG2a
0.3146998604138692
34 Mouse-IgG2b
0.3177544713020325
35 Rat-IgG2b
0.3190629323323568
36 CD20
0.2

NameError: name 'cell_type' is not defined

In [42]:
eval_results = pd.DataFrame(y_pred_eval, index = eval_idx, columns = y_train.columns)
os.chdir('/home/skovtun/Python_projects/Kaggle/Single cell/Single_cell_data/Eval_cite')
eval_results.to_parquet(
    f"X_eval_results.parquet",
    engine="pyarrow",
    index=True,
    coerce_timestamps="ms",
    allow_truncated_timestamps=True,
    version="2.6")
os.chdir(base_dir)

In [43]:
eval_results.head()

gene_id,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
c2150f55becb,0.231686,0.267504,0.812562,3.588033,5.017952,7.736794,3.155045,0.248355,0.590760,0.832704,...,0.251721,5.143633,0.410741,0.332023,0.104586,0.497953,3.120649,0.408277,1.991138,1.720973
65b7edf8a4da,0.253644,0.350562,0.490690,3.818251,4.713515,5.690071,3.244531,0.247756,0.449191,0.428743,...,0.304217,5.043196,0.433041,0.272544,0.083103,0.426059,2.734398,0.421584,2.017819,1.466038
526647a698f8,0.000807,0.217475,0.473151,4.006247,4.708174,4.897419,0.036915,0.199674,0.180025,0.377155,...,-0.000617,3.827174,0.327281,0.093029,0.104951,0.490056,4.978968,0.206673,0.925869,2.015836
ab8f207a3dec,0.083865,0.067022,0.638049,4.622150,5.969405,8.311825,4.564003,0.201294,0.359654,1.026319,...,-0.053421,6.102711,0.500713,-0.025552,0.039365,0.721689,3.279583,0.195828,1.776649,1.911211
08df3dcce25c,0.196436,0.248760,0.496539,3.732250,4.257838,8.610499,4.408142,0.170700,0.529213,0.656816,...,0.227756,5.342353,0.572662,0.180957,0.028210,0.465358,3.274065,0.405459,1.886846,1.688851
